# Spoonacular API Calls

___

### Table Of Contents

1. 
2. 
3. 
4. 
5. 
6. 
7. 
8. 
9. 
10. 

### 1. Import Libraries

In [1]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline
import seaborn as sns
import numpy as np
import pandas as pd
import requests
import json
import sklearn
from scipy import stats
from scipy.stats import norm
from sklearn.utils import resample
import pickle
import statsmodels.api as sm
from statsmodels.formula.api import ols
import scipy.stats as stats
from wordcloud import WordCloud
import random

___

### 2. Expand maximum range of rows

Lots of incoming data from the Spoonacular API.<br>
It is probably a good idea to expand the max range of rows/columns/width to view in Pandas.<br>
Use <b>pd.set_option()</b>

In [2]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)

___

### 3. Understanding the Spoonacular API documentation

Documentation for the Wordtracker API:<br> 
https://spoonacular.com/food-api/docs

#### Identify All GET Endpoints (No PATCH, POST, PUT, or DELETE)

- <b>RECIPES</b>
    - recipes/search
    - /recipes/findByNutrients
    - /recipes/findByIngredients
    - /recipes/complexSearch
    - /recipes/{id}/information
    - /recipes/informationBulk
    - /recipes/{id}/similar
    - /recipes/random
    - /recipes/autocomplete
    - /recipes/{id}/equipmentWidget.json
    - /recipes/{id}/priceBreakdownWidget.json
    - /recipes/{id}/ingredientWidget.json
    - /recipes/{id}/nutritionWidget.json
    - /recipes/{id}/analyzedInstructions
    - /recipes/extract
    - /recipes/{id}/summary
    - /recipes/queries/analyze
    - /recipes/guessNutrition
    
    
- <b>INGREDIENTS</b>
    - /food/ingredients/{id}/information
    - /recipes/convert
    - /food/ingredients/autocomplete
    - /food/ingredients/substitutes
    - /food/ingredients/{id}/substitutes
    
    
- <b>PRODUCTS</b>
    - /food/products/search
    - /food/products/upc/{upc}
    - /food/products/{id}
    - /food/products/upc/{upc}/comparable
    - /food/products/suggest
    
    
- <b>MENU ITEMS</b>
    - /food/menuItems/search
    - /food/menuItems/{id}
    - /food/menuItems/suggest
    
    
- <b>MEAL PLANS</b>
    - /recipes/mealplans/generate
    
    
- <b>WINE</b>
    - /food/wine/dishes
    - /food/wine/pairing
    - /food/wine/description
    - /food/wine/recommendation
    
    
- <b>MISC</b>
    - /recipes/quickAnswer
    - /food/detect
    - /food/site/search
    - /food/videos/search
    - /food/jokes/random
    - /food/trivia/random
    - /food/converse
    - /food/converse/suggest
    
    
- <b>WIDGETS</b>
    - /recipes/{id}/ingredientWidget
    - /recipes/{id}/equipmentWidget
    - /recipes/{id}/priceBreakdownWidget
    - /recipes/{id}/nutritionWidget
    - /recipes/visualizeNutrition
    - /recipes/visualizePriceEstimator
    - /recipes/visualizeEquipment
    - /recipes/visualizeIngredients
    - /food/products/{id}/nutritionWidget
    - /food/menuItems/{id}/nutritionWidget
    - /recipes/visualizeRecipe

#### Identify <u>critical</u> GET endpoints and describe them

- <b>RECIPES</b>
    - <b>recipes/search</b>
        - Spoonacular recipe API includes over 360,000 recipes as well as an open source recipe database. Consider using the "Search Recipes Complex" endpoint for much more flexibility.
    - <b>/recipes/findByNutrients</b>
        - Find a set of recipes that adhere to the given nutritional limits. You may set limits for macronutrients (calories, protein, fat, and carbohydrate) and/or many micronutrients.
    - <b>/recipes/findByIngredients</b>
        - Ever wondered what recipes you can cook with the ingredients you have in your fridge or pantry? This endpoint lets you find recipes that either maximize the usage of ingredients you have at hand (pre shopping) or minimize the ingredients that you don't currently have (post shopping). Find recipes that use as many of the given ingredients as possible and require as few additional ingredients as possible. This is a "what's in your fridge" API endpoint.
    - <b>/recipes/complexSearch</b>
        - Search through hundreds of thousands of recipes using advanced filtering and ranking. NOTE: This method combines searching by query, by ingredients, and by nutrients into one endpoint.
    - <b>/recipes/{id}/information</b>
        - Use a recipe id to get full information about a recipe, such as ingredients, nutrition, diet and allergen information, etc.
    - <b>/recipes/informationBulk</b>
        - Get information about multiple recipes at once. This is equivalent to calling the Get Recipe Information endpoint multiple times, but faster.
    - <b>/recipes/{id}/similar</b>
        - Find recipes which are similar to the given one.
    - <b>/recipes/{id}/summary</b>
        - Automatically generate a short description that summarizes key information about the recipe.
    
    
- <b>INGREDIENTS</b>
    - <b>/food/ingredients/{id}/information</b>
        - Use an ingredient id to get all available information about an ingredient, such as its image and supermarket aisle.
    - <b>/food/ingredients/substitutes</b>
        - Search for substitutes for a given ingredient.
    
    
- <b>PRODUCTS</b>
    - <b>/food/products/search</b>
        - Search packaged food products, such as frozen pizza or Greek yogurt.
    - <b>/food/products/{id}</b>
        - Use a product id to get full information about a product, such as ingredients, nutrition, etc.
    
    
- <b>MENU ITEMS</b>
    - <b>/food/menuItems/search</b>
        - Search over 115,000 menu items from over 800 fast food and chain restaurants. For example, McDonald's Big Mac or Starbucks Mocha.
            - <b>/food/menuItems/{id}</b>
        - Use a menu item id to get all available information about a menu item, such as nutrition.
            - <b>/food/menuItems/suggest</b>
        - Generate suggestions for menu items based on a (partial) query. The matches will be found by looking in the title only.

___

### 4. Test A Call To Spoonacular API

#### Get a recipe

Test a call to the <b>/recipes/search</b> endpoint to make sure we can return a recipe.

In [104]:
response_1_name = requests.get('https://api.spoonacular.com/recipes/search?query=Kale+and+Quinoa+Salad&number=1&apiKey=c6943e3e749f4d5c8627c071b32cd36a')
example_recipe_name = response_1.json()
example_recipe_name

{'results': [{'id': 592479,
   'title': 'Kale and Quinoa Salad with Black Beans',
   'readyInMinutes': 50,
   'servings': 6,
   'image': 'Kale-and-Quinoa-Salad-with-Black-Beans-592479.jpg',
   'imageUrls': ['Kale-and-Quinoa-Salad-with-Black-Beans-592479.jpg']}],
 'baseUri': 'https://spoonacular.com/recipeImages/',
 'offset': 0,
 'number': 1,
 'totalResults': 313853,
 'processingTimeMs': 650,
 'expires': 1579466850458}

#### Get more detailed information about this recipe.

Test a call to the <b>/recipes/{id}/information</b> endpoint to make sure we can use the "id" number provided to return more detailed information about this "Kale and Quinoa Salad with Black Beans" recipe that we got above.

In [39]:
response_1_info = requests.get('https://api.spoonacular.com/recipes/592479/information?includeNutrition=true&apiKey=c6943e3e749f4d5c8627c071b32cd36a')
example_recipe_info = response_1_info.json()
example_recipe_info

{'vegetarian': True,
 'vegan': True,
 'glutenFree': True,
 'dairyFree': True,
 'veryHealthy': True,
 'cheap': False,
 'veryPopular': True,
 'sustainable': False,
 'weightWatcherSmartPoints': 6,
 'gaps': 'no',
 'lowFodmap': False,
 'ketogenic': False,
 'whole30': False,
 'preparationMinutes': 25,
 'cookingMinutes': 25,
 'sourceUrl': 'http://blog.fatfreevegan.com/2013/06/kale-and-quinoa-salad-with-black-beans.html',
 'spoonacularSourceUrl': 'https://spoonacular.com/kale-and-quinoa-salad-with-black-beans-592479',
 'aggregateLikes': 50078,
 'spoonacularScore': 100.0,
 'healthScore': 100.0,
 'creditsText': 'Fat Free Vegan',
 'sourceName': 'Fat Free Vegan',
 'pricePerServing': 101.23,
 'extendedIngredients': [{'id': 1022009,
   'aisle': 'Ethnic Foods;Spices and Seasonings',
   'image': 'chili-powder.jpg',
   'consitency': 'solid',
   'name': 'ancho chile powder',
   'original': '1 teaspoon ancho chile powder (or other pure chile powder, not a blend)',
   'originalString': '1 teaspoon ancho c

#### Explore what kind of information that we can get out of here.

In [40]:
#this should be a dict since we used .json()
type(example_recipe_info)

dict

In [42]:
#what are the keys of our dictionary?
example_recipe_info_keys = list(example_recipe_info.keys())
example_recipe_info_keys

['vegetarian',
 'vegan',
 'glutenFree',
 'dairyFree',
 'veryHealthy',
 'cheap',
 'veryPopular',
 'sustainable',
 'weightWatcherSmartPoints',
 'gaps',
 'lowFodmap',
 'ketogenic',
 'whole30',
 'preparationMinutes',
 'cookingMinutes',
 'sourceUrl',
 'spoonacularSourceUrl',
 'aggregateLikes',
 'spoonacularScore',
 'healthScore',
 'creditsText',
 'sourceName',
 'pricePerServing',
 'extendedIngredients',
 'id',
 'title',
 'readyInMinutes',
 'servings',
 'image',
 'imageType',
 'nutrition',
 'cuisines',
 'dishTypes',
 'diets',
 'occasions',
 'winePairing',
 'instructions',
 'analyzedInstructions']

In [43]:
#The unique id for the recipe.
example_recipe_info['id']

592479

In [45]:
#Nutrition data for the recipe is pretty long, let's just get calories. 
print(example_recipe_info['nutrition']['nutrients'][0]['title'])

#Amount of calories that are in this recipe.
example_recipe_info['nutrition']['nutrients'][0]['amount']

Calories


242.6

In [53]:
#How long it takes to prepare the recipe.
print('Recipe Total Time (In Minutes)')
example_recipe_info['readyInMinutes']

Recipe Total Time (In Minutes)


50

In [47]:
#Number of "likes" for this recipe.
print('Recipe Number Of Likes')
example_recipe_info['aggregateLikes']

Recipe Number Of Likes


50078

In [136]:
#Spoonacular's "score" for a particular recipe.
print('Recipe Spoonacular Score (0-100)')
example_recipe_info['spoonacularScore']

Recipe Spoonacular Score (0-100)


100.0

In [55]:
#Price per serving (in cents)
print('Price Per Serving (In Cents)')
example_recipe_info['pricePerServing']

Price Per Serving (In Cents)


101.23

In [56]:
print('Number Of Servings')
example_recipe_info['servings']

Number Of Servings


6

In [59]:
#It looks like Spoonacular is measuring "price" in cents.
#We need to divide by 100 to get price in dollars.
total_price = example_recipe_info['servings'] * example_recipe_info['pricePerServing']
print('Total Price For Recipe (In Dollars)')
round(total_price / 100,2)

Total Price For Recipe (In Dollars)


6.07

In [119]:
response_1_price = requests.get('https://api.spoonacular.com/recipes/592479/priceBreakdownWidget.json?&apiKey=c6943e3e749f4d5c8627c071b32cd36a')
example_recipe_price = response_1_price.json()
example_recipe_price

{'ingredients': [{'name': 'ancho chile powder',
   'image': 'chili-powder.jpg',
   'price': 24.21,
   'amount': {'metric': {'value': 1.0, 'unit': 'tsp'},
    'us': {'value': 1.0, 'unit': 'tsp'}}},
  {'name': 'avocado',
   'image': 'avocado.jpg',
   'price': 150.0,
   'amount': {'metric': {'value': 1.0, 'unit': ''},
    'us': {'value': 1.0, 'unit': ''}}},
  {'name': 'canned black beans',
   'image': 'black-beans.jpg',
   'price': 39.62,
   'amount': {'metric': {'value': 258.0, 'unit': 'g'},
    'us': {'value': 1.5, 'unit': 'cups'}}},
  {'name': 'carrot',
   'image': 'sliced-carrot.png',
   'price': 11.24,
   'amount': {'metric': {'value': 64.0, 'unit': 'g'},
    'us': {'value': 0.5, 'unit': 'cup'}}},
  {'name': 'chile powder',
   'image': 'chili-powder.jpg',
   'price': 2.14,
   'amount': {'metric': {'value': 0.25, 'unit': 'tsps'},
    'us': {'value': 0.25, 'unit': 'tsps'}}},
  {'name': 'cumin',
   'image': 'ground-cumin.jpg',
   'price': 6.61,
   'amount': {'metric': {'value': 0.5, 'un

In [123]:
#confirm that data is consistent for price - looks good.
round(example_recipe_price['totalCost'] / 100,2)

6.07

In [124]:
#Let's see what the /recipes/{id}/summary endpoint gets us.
#It looks like just a messy verbal description combining all the
#raw data in the /recipes/{id}/information endpoint.

response_1_summary = requests.get('https://api.spoonacular.com/recipes/592479/summary?&apiKey=c6943e3e749f4d5c8627c071b32cd36a')
example_recipe_summary = response_1_summary.json()
example_recipe_summary

{'id': 592479,
 'title': 'Kale and Quinoa Salad with Black Beans',
 'summary': 'If you want to add more <b>gluten free, dairy free, lacto ovo vegetarian, and vegan</b> recipes to your recipe box, Kale and Quinoa Salad with Black Beans might be a recipe you should try. For <b>$1.01 per serving</b>, you get a side dish that serves 6. One portion of this dish contains approximately <b>10g of protein</b>, <b>7g of fat</b>, and a total of <b>243 calories</b>. If you have carrot, vegetable broth, red bell pepper, and a few other ingredients on hand, you can make it. It is brought to you by blog.fatfreevegan.com. This recipe is liked by 50078 foodies and cooks. From preparation to the plate, this recipe takes roughly <b>50 minutes</b>. All things considered, we decided this recipe <b>deserves a spoonacular score of 100%</b>. This score is excellent. Try <a href="https://spoonacular.com/recipes/skip-the-rice-and-try-quinoa-mexican-quinoa-salad-with-black-beans-and-corn-530227">Skip the Rice an

In [134]:
#cleaning up this response ^
example_recipe_summary['summary'].replace('</b>','').replace('<b>','').replace('<', '').replace('>','').replace('//','').replace('href="https:spoonacular.com/recipes/skip-the-rice-and-try-quinoa-mexican-quinoa-salad-with-black-beans-and-corn-530227"','').replace('href="https:spoonacular.com/recipes/black-quinoa-kale-salad-15436"','').replace(' Try a Skip the Rice and Try Quinoa – Mexican Quinoa Salad with Black Beans and Corn/a, a Black Quinoa & Kale Salad/a, and a href="https:spoonacular.com/recipes/black-quinoa-kale-salad-42466"Black Quinoa & Kale Salad/a for similar recipes.','')

'If you want to add more gluten free, dairy free, lacto ovo vegetarian, and vegan recipes to your recipe box, Kale and Quinoa Salad with Black Beans might be a recipe you should try. For $1.01 per serving, you get a side dish that serves 6. One portion of this dish contains approximately 10g of protein, 7g of fat, and a total of 243 calories. If you have carrot, vegetable broth, red bell pepper, and a few other ingredients on hand, you can make it. It is brought to you by blog.fatfreevegan.com. This recipe is liked by 50078 foodies and cooks. From preparation to the plate, this recipe takes roughly 50 minutes. All things considered, we decided this recipe deserves a spoonacular score of 100%. This score is excellent.'

### 5. Call All Project Data From Spoonacular API

#### Get all recipes that we are interested in studying

Which recipes do we want to study?<br>
Which ingredients in particular are we interested in?<br>
What calorie thresholds are we interested in?<br>
...<br>
...<br>
???

In [137]:
#Likely need a "for" loop to make calls here.
def api_call():
    pass